In [4]:
import os
import pandas as pd
import numpy as np
import freesound
from IPython.display import display
from urllib.error import *

FREESOUND_API_KEY = 'qhJktJtXU57oRger9imdCK1vyWhTq9qMQlssaIRa' # Create and add your API key
FILES_DIR = 'Audio_Datasets/augmentation_datasets/noise'  # Place where to store the downloaded diles. Will be relative to the current folder.
FREESOUND_STORE_METADATA_FIELDS = ['id', 'name', 'username', 'previews', 'license', 'tags']  # Freesound metadata properties to store

freesound_client = freesound.FreesoundClient()
freesound_client.set_token(FREESOUND_API_KEY)
if not os.path.exists(FILES_DIR): os.mkdir(FILES_DIR)

In [5]:
def query_freesound(query, filter, num_results=10):
    """Queries freesound with the given query and filter values.
    If no filter is given, a default filter is added.
    """
    if filter is None:
        filter = 'duration:[5 TO 180]'  # Set default filter
    pager = freesound_client.text_search(
        query = query,
        filter = filter,
        fields = ','.join(FREESOUND_STORE_METADATA_FIELDS),
        group_by_pack = 1,
        page_size = num_results
    )
    return [sound for sound in pager]

def retrieve_sound_preview(sound, directory):
    """Download the high-quality wav sound preview of a given Freesound sound object to the given directory.
    """
    return freesound.FSRequest.retrieve(
        sound.previews.preview_hq_ogg,
        freesound_client,
        os.path.join(directory, sound.previews.preview_hq_ogg.split('/')[-1])
    )

In [7]:
# Build our collection of sounds

# Our collection of sounds is made by appending the results of a number of different queries to freesound
# The query terms, query filters and the number of results per query are all defined here.
# Information about how to define filters can be found in the Freesound API documentation: https://freesound.org/docs/api/resources_apiv2.html#request-parameters-text-search-parameters
freesound_queries = [
    {
        'query': 'subway',
        'filter': None ,
        'num_results': 100, 
    },
    {
        'query': 'underground',
        'filter': None,
        'num_results': 100, 
    },
    {
        'query': 'traffic',
        'filter': None,
        'num_results': 100, 
    },
    {
        'query': 'pub',
        'filter': None,
        'num_results': 100, 
    },
    {
        'query': 'classroom',
        'filter': None,
        'num_results': 100, 
    },
    {
        'query': 'street',
        'filter': None,
        'num_results': 100, 
    },
    {
        'query': 'playground',
        'filter': None,
        'num_results': 50, 
    },
    {
        'query': 'crowd',
        'filter': None,
        'num_results': 100, 
    },
    {
        'query': 'metro',
        'filter': None,
        'num_results': 50, 
    },
    {
        'query': 'mall',
        'filter': None,
        'num_results': 100, 
    },
    {
        'query': 'soundscape',
        'filter': None,
        'num_results': 100, 
    },
]

# Do all queries and concatenate the results in a single list of sounds
sounds = sum([query_freesound(query['query'], query['filter'], query['num_results']) for query in freesound_queries],[])

# Download the sounds and save them to FILES_DIR folder
for count, sound in enumerate(sounds):
    print('Downloading sound with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(sounds)))
    try:
        retrieve_sound_preview(sound, FILES_DIR)
    except ContentTooShortError:
        continue